In [ ]:
# importing the libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Importing the dataset into a variable
f = pd.read_csv('G:\Data Sets\germanCredit.data',delim_whitespace = True,header=None)
f.columns=['Current_Account' , 'Duration_in_month', 'Credit_History' , 'Purpose' , 'Credit_Amount' , 'Saving_Account', 'Present_Employment' , 'Instalment_Rate' , 'Status', 'Debtors', 'Residence','Property', 'Age', 'Installment_Plan', 'Housing' , 'Credits_at_bank', 'Job', 'Maintenance' , 'Telephone', 'Foreign_workers', 'Credit_Card_Approval']
f.head()

In [ ]:
# Creating the feature variable
x=f.iloc[:,:20].values
x[1:2]

In [ ]:
#Encoding the given data using LABEL ENCODER

from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
x[:,0]= le.fit_transform(x[:,0])
x[:,2]= le.fit_transform(x[:,2])
x[:,3]= le.fit_transform(x[:,3])
x[:,5]= le.fit_transform(x[:,5])
x[:,6]= le.fit_transform(x[:,6])
x[:,8]= le.fit_transform(x[:,8])
x[:,9]= le.fit_transform(x[:,9])
x[:,11]= le.fit_transform(x[:,11])
x[:,13]= le.fit_transform(x[:,13])
x[:,14]= le.fit_transform(x[:,14])
x[:,16]= le.fit_transform(x[:,16])
x[:,18]= le.fit_transform(x[:,18])
x[:,19]= le.fit_transform(x[:,19])
x[1:2]

In [ ]:
# Importing the data into feature and label variabels
A = x
B= pd.get_dummies(f.Credit_Card_Approval)
B.shape

In [ ]:
# Splitting the data into training and test data
x_train,x_test,y_train,y_test = train_test_split(A,B,test_size=0.3,random_state=0)

In [ ]:
#parameters
lr_rate = 0.01
epochs = 50
batch_size =500

In [ ]:
# neural parameters
num_input = 20
hid_layer1  = 100
hid_layer2 = 50
hid_layer3 = 25
num_classes = 2

In [ ]:
# creating the place holders
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)

In [ ]:
# Store the values of weight(W) and biases(b)
W = {'h1':tf.Variable(tf.random_normal([num_input,hid_layer1])),
     'h2':tf.Variable(tf.random_normal([hid_layer1,hid_layer2])),
     'h3':tf.Variable(tf.random_normal([hid_layer2,hid_layer3])),
    'out':tf.Variable(tf.random_normal([hid_layer3,num_classes]))}

B = {'h1':tf.Variable(tf.random_normal([hid_layer1])),
     'h2':tf.Variable(tf.random_normal([hid_layer2])),
     'h3':tf.Variable(tf.random_normal([hid_layer3])),
    'out':tf.Variable(tf.random_normal([num_classes]))}

In [ ]:
# Defining the layers
def neu_network(X):
    l_1 = tf.add(tf.matmul(X,W['h1']),B['h1'])
    L1 = tf.nn.relu(l_1)
    
    l_2 = tf.add(tf.matmul(l_1,W['h2']),B['h2'])
    L2 = tf.nn.sigmoid(l_2)
    
    l_3 = tf.add(tf.matmul(l_2,W['h3']),B['h3'])
    L3 = tf.nn.sigmoid(l_3)
    
    layer_out = tf.add(tf.matmul(l_3,W['out']),B['out'])
    return layer_out

In [ ]:
#calling the function and calculating the result of the outputlayer
y = neu_network(X)
#y = tf.reduce_sum(y,axis= 1)
y_out = tf.nn.softmax(y)

In [ ]:
# calculating the costfunction
costfun = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y,labels=Y))
#optimiser
opt = tf.train.AdamOptimizer(learning_rate = lr_rate).minimize(costfun)


In [ ]:
#FinalTest Accuracy 
cor_pred = tf.equal(tf.argmax(y_out,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(cor_pred,tf.float32))

In [ ]:
# Initializing global variable initializer
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# Training the data
for epoch in range (epochs):
    sess.run(opt,feed_dict = {X:x_train,Y:y_train})
    lf = sess.run(costfun,feed_dict = {X:x_train ,Y:y_train})
    acc = sess.run(accuracy,feed_dict = {X:x_train ,Y : y_train})
    print("Epoch:",epoch  ,'----','cost:',lf,'----','----','Training Accuracy:',acc)

In [ ]:
#Final mean square error
y_pred = sess.run(y,feed_dict={X:x_test})

# Accuracy of the test data
print("Test Accuracy:",sess.run(accuracy,feed_dict = {X:x_test ,Y:y_test})*100)

In [ ]:
# Calculating the MEAN SQUARED ERROR
mse = tf.reduce_mean(tf.square(y_pred - y_test))
print('MSE: %4f',sess.run(mse))

In [ ]:
# Initializing global variable initializer
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# Training the data
for epoch in range (epochs):
    for i in range (len(x_train)):
        start = i
        end = i + batch_size
        x_train = x_train[start:end]
        y_train = y_train[start:end]
        
        sess.run(opt,feed_dict = {X:x_train,Y:y_train})
        i += batch_size
        lf = sess.run(costfun,feed_dict = {X:x_train ,Y:y_train})
        acc = sess.run(accuracy,feed_dict = {X:x_train ,Y : y_train})
        print("Epoch:",epoch  ,'----','cost:',lf,'----','----','Training Accuracy:',acc)
        
    